<a href="https://colab.research.google.com/github/GIMMI42PIASTRATO/AI-Chess-BOT/blob/main/AI_Chess_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation of all the dependencies

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download arevel/chess-games
!unzip -qq /content/chess-games.zip

100% 1.44G/1.45G [00:15<00:00, 161MB/s]
100% 1.45G/1.45G [00:15<00:00, 100MB/s]


In [ ]:
!pip install chess -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00


#CODE

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import chess
import numpy as np
import pandas as pd
from pandas.core.generic import gc
import re

##Mapping the chess board

In [ ]:
letter_to_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_to_letter = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

##Chess board to matrix

###Board to rappresentation
This function create a numpy tensor (3D numpyarray) which rapresent the board containing every position of every chess pice.
Every chess pice has is own mapping (Is a matrix representing the chessboard that maps the position of only one type of pieces)

In [ ]:
def board_to_rep(board: chess.Board()):
  pieces = ['p', 'r', 'n', 'b', 'q', 'k'] #Initials of the names of the pieces
  layers = []
  for piece in pieces:
    layers.append(layer_to_rep(board, piece))
  board_rep = np.stack(layers)
  return board_rep

###Layer rappresentation

In [ ]:
def layer_to_rep(board, piece):
  str_board = str(board)
  str_board = re.sub(f"[^{piece}{piece.upper()} \n]", ".", str_board)
  str_board = re.sub(f"{piece}", "1", str_board)
  str_board = re.sub(f"{piece.upper()}", "-1", str_board)
  str_board = re.sub("\.", "0", str_board)

  board_matrix = []
  for row in str_board.split("\n"):
    row = row.split()
    row = [int(x) for x in row]
    board_matrix.append(row)

  return np.array(board_matrix)

##Chess move to matrix

###Move to rappresentation

In [ ]:
def move_to_rep(move, board: chess.Board()):
  board.push_san(move).uci()
  move = str(board.pop())

  from_output_layer = np.zeros((8, 8))
  from_row = 8 - int(move[1])
  from_column = letter_to_num[move[0]]
  from_output_layer[from_row][from_column] = 1

  to_output_layer = np.zeros((8, 8))
  to_row = 8 - int(move[3])
  to_column = letter_to_num[move[2]]
  to_output_layer[to_row][to_column]

  return np.stack((from_output_layer, to_output_layer))

##Create move list from dataset move list


In [ ]:
def create_move_list(string_move_list):
    return re.sub("\d*\. ", '', string_move_list).split()[:-1]

#Importing and filtering the data

In [ ]:
chess_data_raw = pd.read_csv("/content/chess_games.csv", usecols=["AN", "WhiteElo", "BlackElo"])

In [ ]:
chess_data = chess_data_raw[(chess_data_raw["WhiteElo"] >= 1900) & (chess_data_raw["BlackElo"] >= 1900)]
del chess_data_raw
gc.collect()
chess_data = chess_data[["AN"]]
chess_data = chess_data[~chess_data["AN"].str.contains("{")]
chess_data = chess_data[chess_data["AN"].str.len() > 20]
print(chess_data.shape)

(1017420, 1)


In [ ]:
chess_data.head()

,AN
4,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
7,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
8,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
13,1. b4 e6 2. Bb2 d5 3. Nf3 Nf6 4. b5 Be7 5. e3 ...
16,1. e4 c6 2. Nf3 d5 3. Nc3 g6 4. d3 Bg7 5. Be2 ...


#Creating the Dataset
In PyTorch, la classe Dataset è utilizzata per gestire e organizzare i dati che verranno utilizzati per l'addestramento e la validazione dei modelli di apprendimento automatico. Questi dati possono essere immagini, testo, suoni o qualsiasi altro tipo di informazione. La classe Dataset fornisce un'interfaccia comoda per accedere a questi dati in modo efficiente.

Il metodo __getitem__ è un metodo speciale nella classe Dataset che ti permette di accedere a un singolo campione di dati in base al suo indice. Quando crei una sottoclasse della classe Dataset e implementi il metodo __getitem__, stai fondamentalmente definendo come ottenere un campione specifico dai dati. Ad esempio, se stai lavorando con un set di immagini, __getitem__ potrebbe caricare un'immagine dal disco e applicare le trasformazioni necessarie.

In [42]:
class ChessDataset(Dataset):

    def __init__(self, games):
        super().__init__()
        self.games = games

    def __len__(self):                                          #len(self) return call __len__
        return self.games.shape[0]

    def __getitem__(self, key):
        game_index = np.random.randint(self.games.shape[0])     #Create the index for get the game
        random_game = self.games['AN'].values[game_index]       #Get the game using the game_index
        moves = create_move_list(random_game)                   #Create the move list using the create_move_list function
        move_index = np.random.randint(len(moves) - 1)          #Create the index for get the move
        next_move = moves[move_index]                           #Get the move
        moves = moves[:move_index]                              #I don't know why this
        board = chess.Board()                                   #Create the board object

        for move in moves:
            board.push_san(move)

        board_rep = board_to_rep(board)
        move_rep = move_to_rep(next_move, board)

        if move_index % 2:
            board_rep *= -1
        return board_rep, move_rep


##Initialisation of the dataset

In [43]:
data_train = ChessDataset(chess_data['AN'])
data_train_loader = DataLoader(data_train, batch_size=32, shuffle=True, drop_last=True)

#Creating the neural network

In [44]:
class Module(nn.Module):

    def __init__(self, hidden_layer):
       super().__init__()
       self.conv1 = nn.Conv2d

NameError: ignored